In [1]:
from collections import Counter
from pprint import pprint
from tqdm import tqdm

import json 
import re

In [2]:
data = json.load(open('/Users/nt13157/Downloads/result(1).json', 'r'))

pprint(data[0])

{'diagnosis': 'MCI',
 'dx1': 610,
 'participant_id': 120,
 'question': 'Describe what is going on in the picture.',
 'repeat': 0,
 'response': ['okay ',
              'the boy is getting cookies from the cookie jar and handing one '
              '<to his> [//] to the girl ',
              'oh and the woman is wiping the dishes with her back to the '
              'children ',
              'and the sink is running over and spilling water onto the floor ',
              "I guess that's all the action ",
              'oh no the [/] the stool &=laughs is [/] &ti is tipping over ',
              '&=laughs the stool that the boy is standing on is tipping over ',
              'that appears to be all the action ',
              'mhm ']}


In [3]:
pprint(Counter([datum['diagnosis'] for datum in data]))
pprint(len(data))

Counter({'ProbableAD': 178,
         'Control': 165,
         'MCI': 21,
         'PossibleAD': 21,
         'Vascular': 5,
         'Complains': 2,
         'Parkinson': 1})
393


In [4]:
scale = dict(
    Control=0, 
    PossibleAD=1, 
    ProbableAD=2, 
    Vascular=3,
)

data = [datum for datum in data if datum['diagnosis'] in scale.keys()]

len(data)

369

In [16]:
fill = '[^>]+'

replacement_patterns = [
        '<' + fill + '>', 
    r'\[\:' + fill + '\]', 
    r'\[\*' + fill + '\]', 
    r'\[\+' + fill + '\]', 
    r'\[/+\]', 
]

repetition_patterns = [
    r'\((.*?)\)',
]

time_pattern = u'\x15' + fill + u'\x15'

X = []
y = []

for di, datum in enumerate(tqdm(data[:])): 
    sentence = ''
    
    for response in datum['response']: 
        response = response.strip().lower()
        
        if response == '(': 
            continue

        response = re.sub(time_pattern, '', response.strip())

        for replacement_pattern in replacement_patterns: 
            response = re.sub(replacement_pattern, '', response.strip())

        for repetition_pattern in repetition_patterns: 
            response = re.sub(repetition_pattern, '', response.strip())

        for punctuation in '.,': 
            response = response.replace(punctuation, ' {} '.format(punctuation))

        while '  ' in response: 
            response = response.replace('  ', ' ')
            
        sentence = f'{sentence} {response}'
        
    X.append(sentence)
    y.append(scale[datum['diagnosis']])

i = 10

pprint(data[i])
pprint(X[i])        

json.dump(X, open('data.json', 'w'), indent=2)
json.dump(y, open('labels.json', 'w'), indent=2)

100%|██████████| 369/369 [00:00<00:00, 6953.88it/s]

{'diagnosis': 'ProbableAD',
 'dx1': 101,
 'participant_id': 148,
 'question': 'Describe what is going on in the picture.',
 'repeat': 0,
 'response': ['well <the table [: stool] [* s:r-ret]> [//] the seat [: stool] '
              '[* s:r] is falling ',
              'the sink is running over ',
              "the girl's reaching for a cookie ",
              'the mom is dryin(g) a dish ',
              '&hm &hm cup and saucers there ',
              '(',
              "I <didn't hit> [//] did this and then I did that ",
              "&hm well yeah here's some outside (",
              '&uh a garden I guess ']}
(" well the seat is falling the sink is running over the girl's reaching for a "
 'cookie the mom is dryin a dish &hm &hm cup and saucers there i did this and '
 "then i did that &hm well yeah here's some outside ( &uh a garden i guess")
